 # Cell 1 – Load Parsed Sessions from Pickle File
  This cell loads the parsed SLAM sessions (slam_sessions.pkl) from the
  data/interim folder.
  These are the raw token-level session logs we will use to extract metadata such
  as user ID, session type, client platform, time spent, and activity days

In [6]:
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
from typing import List, Dict


# Set project root (assuming current script is in a subfolder)
project_root = Path.cwd().parent

# Define path to the pickle file
parsed_sessions_path = project_root / "data" / "interim" / "slam_sessions.pkl"

# Load the pickle file
with open(parsed_sessions_path, "rb") as f:
    slam_sessions = pickle.load(f)

print(f"✅ slam_sessions loaded. Total sessions: {len(slam_sessions)}")


✅ slam_sessions loaded. Total sessions: 824012


 # Cell 2 – Extract Metadata to DataFrame
 This cell extracts key metadata from each session such as user_id,
 session_type, client, days, time, etc.
 We create a structured DataFrame df_sessions and handle missing/null values
 for numeric fields

In [7]:

def extract_metadata(session: List[str]) -> Dict[str, any]:
    """
    Extract metadata from session lines.
    
    Args:
        session (List[str]): List of session lines
        
    Returns:
        Dict[str, any]: Extracted metadata containing:
            - prompt: String containing the session prompt
            - user_id: Unique identifier for the user
            - countries: Country codes
            - days: Number of days since first activity
            - client: Platform used (web/android/ios)
            - session_type: Type of learning session
            - format: Session format
            - time: Time spent in session
    """
    metadata = {
        "prompt": "", "user_id": "", "countries": "", "days": 0,
        "client": "", "session_type": "", "format": "", "time": 0
    }
    # Extract metadata from session lines
    for line in session:
        if line.startswith("# prompt:"):
            metadata["prompt"] = line.replace("# prompt:", "").strip()
        elif line.startswith("# user:"):
            parts = line.replace("# user:", "").strip().split()
            metadata["user_id"] = parts[0]
            for part in parts[1:]:
                if ":" in part:
                    key, value = part.split(":")
                    if key == "session":
                        metadata["session_type"] = value
                    elif key == "days":
                        metadata["days"] = value
                    elif key in metadata:
                        metadata[key] = value
    return metadata

# Apply extractor to sessions that have prompts
parsed_metadata = [
    extract_metadata(s) for s in slam_sessions
    if any("# prompt:" in line for line in s)
]

# Build DataFrame
df_sessions = pd.DataFrame(parsed_metadata)

# Clean up 'time' and 'days' columns
df_sessions['time'] = pd.to_numeric(df_sessions['time'].replace(['null', '', None], 0))
df_sessions['days'] = pd.to_numeric(df_sessions['days'].replace(['null', '', None], 0))

# Add validation function
def validate_features(df: pd.DataFrame) -> bool:
    """
    Validate engineered features for data quality.
    
    Args:
        df (pd.DataFrame): DataFrame containing session features
        
    Returns:
        bool: True if all validations pass, False otherwise
    """
    checks = {
        "No nulls in key fields": df[['user_id', 'time', 'days']].notnull().all().all(),
        "Positive time values": (df['time'] >= 0).all(),
        "Valid day range": (df['days'] >= 0).all() & (df['days'] <= 14).all(),
        "Valid user_ids": df['user_id'].astype(str).str.len().gt(0).all(),
        "Session types present": df['session_type'].notna().any(),
        "Client types present": df['client'].notna().any()
    }
    
    print("\n🔍 Data Validation Results:")
    for check, result in checks.items():
        print(f"{'✅' if result else '❌'} {check}")
    
    if not all(checks.values()):
        print("\n⚠️ Warning: Some validations failed!")
        failed_checks = [check for check, result in checks.items() if not result]
        print("Failed checks:", failed_checks)
    
    return all(checks.values())

# Run validation
validation_result = validate_features(df_sessions)

# Additional statistics if validation passes
if validation_result:
    print("\n📊 Data Quality Metrics:")
    print(f"Total Sessions: {len(df_sessions):,}")
    print(f"Unique Users: {df_sessions['user_id'].nunique():,}")
    print(f"Date Range: {df_sessions['days'].min():.0f} to {df_sessions['days'].max():.0f} days")
    print(f"Average Session Time: {df_sessions['time'].mean():.2f}")

# Save to CSV
output_path = project_root / "data" / "processed" / "parsed_sessions.csv"
output_path.parent.mkdir(parents=True, exist_ok=True)
df_sessions.to_csv(output_path, index=False)

# Print summary
print(f"\n📁 Metadata saved to: {output_path}")
print(f"✅ Total rows saved: {len(df_sessions)}\n")

# Inspect value counts
print("📊 session_type breakdown:\n", df_sessions['session_type'].value_counts())
print("\n🖥️ client breakdown:\n", df_sessions['client'].value_counts())
print("\n📈 Numeric stats:\n", df_sessions[['days', 'time']].describe())



🔍 Data Validation Results:
✅ No nulls in key fields
❌ Positive time values
❌ Valid day range
✅ Valid user_ids
✅ Session types present
✅ Client types present

⚠️ Warning: Some validations failed!
Failed checks: ['Positive time values', 'Valid day range']

📁 Metadata saved to: f:\Bachleros Research\Rsearch thesis\Predicting-Churn-using-ML-and-DL\data\processed\parsed_sessions.csv
✅ Total rows saved: 595100

📊 session_type breakdown:
 session_type
lesson      493236
practice     93907
test          7957
Name: count, dtype: int64

🖥️ client breakdown:
 client
android    416645
ios        107590
web         70865
Name: count, dtype: int64

📈 Numeric stats:
                 days           time
count  595100.000000  595100.000000
mean        6.072470      24.693122
std         5.631776     766.344798
min         0.000000    -156.000000
25%         1.307000       5.000000
50%         4.355500       9.000000
75%         9.284000      17.000000
max        28.042000  330554.000000


# Cell 3 – Inspect Raw Lines with Session-Type Information
 This helps verify that session types like 
 correctly tagged in the logs

In [8]:
# -------------------- CELL 3: Inspect Session Types -------------------
print("🔍 Checking session lines that might contain 'session_type':\n")
for session in slam_sessions[:10]:
    print("🆕 New session:")
    for line in session:
        if "session_type" in line.lower() or "session:" in line.lower():
            print("  ", line)


🔍 Checking session lines that might contain 'session_type':

🆕 New session:
   # user:XEinXf5+  countries:CO  days:0.003  client:web  session:lesson  format:reverse_translate  time:9
🆕 New session:
   # user:XEinXf5+  countries:CO  days:0.005  client:web  session:lesson  format:reverse_translate  time:12
🆕 New session:
   # user:XEinXf5+  countries:CO  days:0.008  client:web  session:lesson  format:reverse_translate  time:6
🆕 New session:
   # user:XEinXf5+  countries:CO  days:0.008  client:web  session:lesson  format:reverse_translate  time:13
🆕 New session:
   # user:XEinXf5+  countries:CO  days:0.008  client:web  session:lesson  format:reverse_translate  time:16
🆕 New session:
   # user:XEinXf5+  countries:CO  days:0.011  client:web  session:lesson  format:reverse_translate  time:10
🆕 New session:
   # user:XEinXf5+  countries:CO  days:0.011  client:web  session:lesson  format:reverse_translate  time:5
🆕 New session:
   # user:XEinXf5+  countries:CO  days:0.016  client:web  session:

# Cell 4 – Filter Early Sessions (≤14 Days)
 We focus only on sessions from the first 14 days of activity per user.
 This matches our proposal’s scope for predicting churn based on early behavior.

In [9]:
# -------------------- CELL 4: Filter Early Sessions -------------------
df_cleaned = df_sessions.dropna(subset=["time"])
df_cleaned = df_cleaned[df_cleaned["time"] > 0]

# IQR filtering for outliers in time
Q1 = df_cleaned["time"].quantile(0.25)
Q3 = df_cleaned["time"].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df_cleaned = df_cleaned[(df_cleaned["time"] >= lower_bound) & (df_cleaned["time"] <= upper_bound)]

# Normalize session_type
df_cleaned["session_type"] = df_cleaned["session_type"].astype(str).str.lower()

# Filter early sessions
early_sessions = df_cleaned[df_cleaned['days'] <= 14].copy()
print(f"✅ Filtered early sessions: {early_sessions.shape[0]} rows")


✅ Filtered early sessions: 481271 rows


## 🧮 Cell 5 – Feature Engineering: Aggregate Session Data Per User

We create user-level features to summarize behavior across sessions. These include:

- Average session time, total time, and count of sessions
- First and last active day, and list of unique active days
- Counts of different `session_type` and `client` (platforms)

These features are essential for churn modeling and behavioral analysis.


In [10]:
# Group session data per user to extract behavior features
user_features = early_sessions.groupby('user_id').agg({
    'time': ['mean', 'sum', 'count'],
    'days': ['min', 'max', lambda x: sorted(x.unique())],
    'session_type': lambda x: x.value_counts().to_dict(),
    'client': lambda x: x.value_counts().to_dict()
}).reset_index()

# Flatten MultiIndex column names
user_features.columns = [
    'user_id', 'avg_time', 'total_time', 'session_count',
    'first_day', 'last_day', 'active_days',
    'session_type_counts', 'client_counts'
]

print("✅ User features created:")
user_features.head()

def print_feature_stats(df: pd.DataFrame):
    """Print comprehensive feature statistics."""
    print("\n📊 Feature Engineering Summary:")
    print(f"Total Users: {len(df):,}")
    print(f"Average Sessions per User: {df['session_count'].mean():.2f}")
    print(f"Median Total Time: {df['total_time'].median():.2f}")
    
    print("\n📱 Platform Usage:")
    platform_stats = pd.DataFrame([x for x in df['client_counts']])
    print(platform_stats.sum().sort_values(ascending=False))
    
    print("\n📘 Session Types:")
    session_stats = pd.DataFrame([x for x in df['session_type_counts']])
    print(session_stats.sum().sort_values(ascending=False))

# Call the function
print_feature_stats(user_features)

# Save engineered features
features_path = project_root / "data" / "processed" / "user_features.csv"
user_features.to_csv(features_path, index=False)
print(f"\n✅ Features saved to: {features_path}")

✅ User features created:

📊 Feature Engineering Summary:
Total Users: 2,580
Average Sessions per User: 186.54
Median Total Time: 1717.00

📱 Platform Usage:
android    343655.0
ios         79833.0
web         57783.0
dtype: float64

📘 Session Types:
lesson      411222.0
practice     63666.0
test          6383.0
dtype: float64

✅ Features saved to: f:\Bachleros Research\Rsearch thesis\Predicting-Churn-using-ML-and-DL\data\processed\user_features.csv


# 📋 Data Processing Summary

## Quality Checks
- Removed null values
- Filtered outliers using IQR
- Normalized session types
- Limited to first 14 days

## Feature Set
- Time-based: avg_time, total_time, session_count
- Activity: first_day, last_day, active_days
- Behavior: session_type_counts, client_counts

## Next Steps
- Feature correlation analysis
- Handle categorical variables
- Scale numeric features
- Split train/test sets